In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


- feed the features as sequences (sentences or documents?) 
- sequences must be equal length: pad with 0s
- randomized padding proportion?
- could remove title-only documents as they may cause a bias (good idea for the project in general? restrict to abstract text?)

In [2]:
data = pd.read_parquet('data/split/train.parquet')
# OR
data = pd.read_parquet('data/split/train_1000.parquet')

In [ ]:
data_ds

In [5]:
data.unstack().fillna(0).stack()

PM_0      PM_1      PM_2      PM_3      PM_4      PM_5  \
doc      idx                                                               
10036953 0    0.214328  0.087971 -0.264222 -0.168883  0.036122 -0.015450   
         1   -0.095692 -0.195316 -0.128377 -0.040698 -0.262727  0.468707   
         2    0.221656  0.123618 -0.561031  0.257037  0.234218  0.560647   
         3    0.235648  0.001998 -0.415426  0.134422 -0.027661  0.068912   
         4   -0.162358 -0.115302 -0.180101 -0.443206 -0.327403  0.399859   
...                ...       ...       ...       ...       ...       ...   
9989713  833  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
         834  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
         835  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
         836  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
         837  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

                  PM_6      PM_7      PM_8      PM_9  ...  POS_LAG_1_VBN  \
doc      idx                                          ...                  
10036953 0   -1.134604 -0.001892  0.461346  0.234333  ...            0.0   
         1   -0.035658  0.189616  0.588961 -0.260077  ...            0.0   
         2   -0.470945  0.471395  0.543297  0.169361  ...            0.0   
         3    0.265036  0.251053  1.079644  0.424286  ...            0.0   
         4   -0.602110  0.172364  0.729531 -0.292045  ...            0.0   
...                ...       ...       ...       ...  ...            ...   
9989713  833  0.000000  0.000000  0.000000  0.000000  ...            0.0   
         834  0.000000  0.000000  0.000000  0.000000  ...            0.0   
         835  0.000000  0.000000  0.000000  0.000000  ...            0.0   
         836  0.000000  0.000000  0.000000  0.000000  ...            0.0   
         837  0.000000  0.000000  0.000000  0.000000  ...            0.0   

              POS_LAG_1_VBP  POS_LAG_1_VBZ  POS_LAG_1_WDT  POS_LAG_1_WP  \
doc      idx                                                              
10036953 0              0.0            0.0            0.0           0.0   
         1              0.0            0.0            0.0           0.0   
         2              0.0            0.0            0.0           0.0   
         3              0.0            0.0            0.0           0.0   
         4              0.0            1.0            0.0           0.0   
...                     ...            ...            ...           ...   
9989713  833            0.0            0.0            0.0           0.0   
         834            0.0            0.0            0.0           0.0   
         835            0.0            0.0            0.0           0.0   
         836            0.0            0.0            0.0           0.0   
         837            0.0            0.0            0.0           0.0   

              POS_LAG_1_WP$  POS_LAG_1_WRB  interventions  outcomes  \
doc      idx                                                          
10036953 0              0.0            0.0            0.0       0.0   
         1              0.0            0.0            0.0       0.0   
         2              0.0            0.0            0.0       0.0   
         3              0.0            0.0            0.0       0.0   
         4              0.0            0.0            0.0       0.0   
...                     ...            ...            ...       ...   
9989713  833            0.0            0.0            0.0       0.0   
         834            0.0            0.0            0.0       0.0   
         835            0.0            0.0            0.0       0.0   
         836            0.0            0.0            0.0       0.0   
         837            0.0            0.0            0.0       0.0   

              participants  
doc      idx                
10036953 0             0.0  
         1             0.0  
         2             0

In [6]:
data

PM_0      PM_1      PM_2      PM_3      PM_4      PM_5  \
doc      idx                                                               
10036953 0    0.214328  0.087971 -0.264222 -0.168883  0.036122 -0.015450   
         1   -0.095692 -0.195316 -0.128377 -0.040698 -0.262727  0.468707   
         2    0.221656  0.123618 -0.561031  0.257037  0.234218  0.560647   
         3    0.235648  0.001998 -0.415426  0.134422 -0.027661  0.068912   
         4   -0.162358 -0.115302 -0.180101 -0.443206 -0.327403  0.399859   
...                ...       ...       ...       ...       ...       ...   
9989713  289 -0.482327 -0.020039  0.253585 -0.096272  0.212909  0.478674   
         290  0.272813  0.066905 -0.238704 -0.165099  0.000402  0.109383   
         291 -0.116538  0.463041 -0.158457  0.337768 -0.209759  0.854119   
         292 -1.272327 -0.116690  0.162349 -0.350024 -0.083843  0.396636   
         293 -0.269651 -0.169214 -0.238500 -0.227601 -0.297280 -0.135840   

                  PM_6      PM_7      PM_8      PM_9  ...  POS_LAG_1_VBN  \
doc      idx                                          ...                  
10036953 0   -1.134604 -0.001892  0.461346  0.234333  ...              0   
         1   -0.035658  0.189616  0.588961 -0.260077  ...              0   
         2   -0.470945  0.471395  0.543297  0.169361  ...              0   
         3    0.265036  0.251053  1.079644  0.424286  ...              0   
         4   -0.602110  0.172364  0.729531 -0.292045  ...              0   
...                ...       ...       ...       ...  ...            ...   
9989713  289  0.159902  0.234866  1.577489 -0.230714  ...              0   
         290 -0.587040 -0.276197  0.278902 -0.087827  ...              1   
         291 -0.224265  0.123816  0.603451  0.353388  ...              0   
         292 -0.026688 -0.275437  1.399622 -0.381075  ...              0   
         293 -0.380515  0.001409  0.249152 -0.106978  ...              0   

              POS_LAG_1_VBP  POS_LAG_1_VBZ  POS_LAG_1_WDT  POS_LAG_1_WP  \
doc      idx                                                              
10036953 0                0              0              0             0   
         1                0              0              0             0   
         2                0              0              0             0   
         3                0              0              0             0   
         4                0              1              0             0   
...                     ...            ...            ...           ...   
9989713  289              0              0              0             0   
         290              0              0              0             0   
         291              0              0              0             0   
         292              0              0              0             0   
         293              0              0              0             0   

              POS_LAG_1_WP$  POS_LAG_1_WRB  interventions  outcomes  \
doc      idx                                                          
10036953 0                0              0              0         0   
         1                0              0              0         0   
         2                0              0              0         0   
         3                0              0              0         0   
         4                0              0              0         0   
...                     ...            ...            ...       ...   
9989713  289              0              0              0         0   
         290              0              0              0         0   
         291              0              0              0         0   
         292              0              0              0         0   
         293              0              0              0         0   

              participants  
doc      idx                
10036953 0               0  
         1               0  
         2              

In [7]:
# probably simpler on sentence level. Less padding required. Remove excessively long sentences, etc.

In [8]:
data = pd.read_pickle('data/split/train.pkl')
# cols = data.columns.str.startswith('PM')

In [9]:
pio = {'interventions', 'outcomes', 'participants'}
features = set(data.columns).difference(pio)

In [10]:
k_folds = 5 # only simulated for now

# double split, replace with KCV
train_idx, val_idx = train_test_split(data.index.unique('doc'), 
                                      train_size=1-(1/k_folds)) # default 70:30
print(f'splitting data {len(train_idx)}:{len(val_idx)}')

train_set = data.loc[(train_idx, slice(None)),:]
val_set   = data.loc[(train_idx, slice(None)),:]

X_train, X_test = train_set[features], val_set[features]
y_train, y_test = train_set[pio], val_set[pio]
# copies = [deepcopy(data) for data in [X_train, y_train, X_test, y_test]]



splitting data 3600:900


# Model

In [11]:
# set params
# use sent split data?
seq_length = X_train.groupby('doc').count().max(axis=0)[0]
n_features = len(X_train.columns)

input_shape = (seq_length, n_features)

try:
    for col in data.columns:
        
data.groupby('doc').count().max(axis=0)[0]

IndentationError: expected an indented block (<ipython-input-11-6f14f811c1af>, line 11)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

max_features = 1024

model = Sequential()
# model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(X_test, y_test, batch_size=16)

Multi-layer perceptron for binary classification. Could try adapting this as well.

In [ ]:

# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# # Generate dummy data
# X_train = np.random.random((1000, 20))
# y_train = np.random.randint(2, size=(1000, 1))
# X_test = np.random.random((100, 20))
# y_test = np.random.randint(2, size=(100, 1))

# MLP = Sequential()
# model.add(Dense(64, input_dim=20, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           epochs=20,
#           batch_size=128)
# score = model.evaluate(x_test, y_test, batch_size=128)

Bidirectional LSTM for sequences (sentence-wise). Most potential.

In [ ]:
# define problem properties
n_timesteps = 10
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(1000):
	# generate new random sequence
	X,y = get_sequence(n_timesteps)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=1, verbose=2)
# evaluate LSTM
X,y = get_sequence(n_timesteps)
yhat = model.predict_classes(X, verbose=0)
for i in range(n_timesteps):
	print('Expected:', y[0, i], 'Predicted', yhat[0, i])